In [1]:
# Imports
import numpy as np
from pbarlike import *
from pbarlike.drn_interface import create_DIFF_BRK_parameters,create_INJ_BRK_parameters
from pbarlike.data import ams02Data
from pbarlike.propagation import Propagation
from pbarlike.drn_interface import DRNet
from pbarlike.solar_modulation import ForceFieldApprox
from pbarlike.likelihoods import LogLikes,LogLikeRatios

In [2]:
## Global Params
pp_marg = np.array([0.0])
include_low_energy_arg = False
production_xsection_cov_arg = True
pe_arg = True
verbose_arg=False

## Params that give DM preference for DIFF.BRK _ b bbar + marginalization
m_DM_xc  = np.array([10**(5.018181818181818 - 3)])
b_bbar_bf = np.array([1.000e-05, 1.000e-05, 9.993e-01, 1.000e-05, 1.000e-05, 1.000e-05,  1.000e-05, 1.000e-05])
sv_xc = 5.3366992312063e-27
pmdb = "DIFF.BRK"
ppdbmarg = np.genfromtxt('pbarlike/dependencies/'+ pmdb +'/multinest_sample.dat')
pp_db_xc = np.array(ppdbmarg[:,:10])

# Setting params for check
xcheck_db = [m_DM_xc,b_bbar_bf,sv_xc,pp_db_xc,pmdb]
timer_db = [m_DM_xc,b_bbar_bf,sv_xc,pp_marg,pmdb]
[DM_mass,bf,sigma_v,prop_params,prop_model] = xcheck_db

In [3]:
## Loading data, covariance matrices and multinest sample for marginalization;
## for use in later timing sections
data = ams02Data(include_low_energy_arg)
propagation_config = Propagation(prop_model, production_xsection_cov_arg, include_low_energy_arg)
drn = DRNet(propagation_config,prop_params,pe_arg,verbose_arg)
sol_mod = ForceFieldApprox(data, propagation_config,drn,verbose_arg)
chi_squares = LogLikes(data,propagation_config,verbose_arg)
loglike_ratios = LogLikeRatios(drn,chi_squares,sol_mod,verbose_arg)
drn.preprocessing_DMparams(DM_mass, bf, sigma_v)
phi_DM_LIS = drn.DM_LIS_sim()


 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 


In [4]:
%%timeit
## Loading data, covariance matrices and multinest sample for marginalization
data = ams02Data(include_low_energy_arg)
propagation_config = Propagation(prop_model, production_xsection_cov_arg, include_low_energy_arg)

474 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
## Simulating secondary antiproton flux at the interstellar region
drn = DRNet(propagation_config,prop_params,pe_arg,verbose_arg)


 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been init

In [6]:
%%timeit
## Applying solar modulation
sol_mod = ForceFieldApprox(data, propagation_config,drn,verbose_arg)

5.15 s ± 333 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
## Simulating primary antiproton flux at the interstellar region
drn.preprocessing_DMparams(DM_mass, bf, sigma_v)
phi_DM_LIS = drn.DM_LIS_sim()

3.73 s ± 652 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
## Simulating total antiproton flux at the top of the atmosphere; phi_DMCR_uc and phi_DMCR_c are the same
## for marginalization
drn = DRNet(propagation_config,prop_params,pe_arg,verbose_arg)
sol_mod = ForceFieldApprox(data, propagation_config,drn,verbose_arg)
drn.preprocessing_DMparams(DM_mass, bf, sigma_v)
phi_DM_LIS = drn.DM_LIS_sim()
phi_DMCR_uc = sol_mod.TOA_sim(phi_DM_LIS,'uncorrelated')


 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been init

In [9]:
%%timeit
## Calculating uncorrelated log-likelihood ratios
phi_DMCR_uc = sol_mod.TOA_sim(phi_DM_LIS,'uncorrelated')
del_chi2_uncorr = loglike_ratios.del_chi2_uncorr(phi_DMCR_uc)

2.76 s ± 531 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
## Calculating correlated log-likelihood ratios
phi_DMCR_c = sol_mod.TOA_sim(phi_DM_LIS,'correlated')
del_chi2_corr = loglike_ratios.del_chi2_corr(phi_DMCR_c)

2.78 s ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
## A full routine of simulating antiproton fluxes and calculating both uncorrelated log-likelihood ratio
drn = DRNet(propagation_config,prop_params,pe_arg,verbose_arg)
sol_mod = ForceFieldApprox(data, propagation_config,drn,verbose_arg)
chi_squares = LogLikes(data,propagation_config,verbose_arg)
loglike_ratios = LogLikeRatios(drn,chi_squares,sol_mod,verbose_arg)
drn.preprocessing_DMparams(DM_mass, bf, sigma_v)
phi_DM_LIS = drn.DM_LIS_sim()
phi_DMCR_uc = sol_mod.TOA_sim(phi_DM_LIS,'uncorrelated')
del_chi2_uncorr = loglike_ratios.del_chi2_uncorr(phi_DMCR_uc)


 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been init

In [12]:
%%timeit
## A full routine of simulating antiproton fluxes and calculating both uncorrelated log-likelihood ratio
drn = DRNet(propagation_config,prop_params,pe_arg,verbose_arg)
sol_mod = ForceFieldApprox(data, propagation_config,drn,verbose_arg)
chi_squares = LogLikes(data,propagation_config,verbose_arg)
loglike_ratios = LogLikeRatios(drn,chi_squares,sol_mod,verbose_arg)
drn.preprocessing_DMparams(DM_mass, bf, sigma_v)
phi_DM_LIS = drn.DM_LIS_sim()
phi_DMCR_c = sol_mod.TOA_sim(phi_DM_LIS,'correlated')
del_chi2_corr = loglike_ratios.del_chi2_corr(phi_DMCR_c)


 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been initiated. 

 Initializing DRN ...

 At least one of the inputs for gamma_2,p is outside the trained parameter ranges. The default multinest sample will be used for marginalization. 
2.3083323198377546 2.3702775768338533

 The simulation tool has been init